In [2]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
#from linearmodels.iv import IV2SLS


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]
data.head(20)

,ID,Year,Sales,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,const
8,1,2021,257,0.010319,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
9,1,2022,183,0.005938,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
10,1,2023,177,0.002822,Aiways,U5,400,3.308258e+05,201,34,SUV,C,CH,1.0
21,2,2023,3,0.000048,Aiways,U6,405,3.757100e+05,214,34,SUV,C,CH,1.0
28,3,2019,222,0.040217,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
29,3,2020,491,0.034512,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
30,3,2021,260,0.010440,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
31,3,2022,538,0.017456,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
32,3,2023,68,0.001084,Audi,e-tron,375,7.149530e+05,402,17,SUV,F,DE,1.0
41,4,2021,84,0.003373,Audi,e-tron GT,472,1.081416e+06,522,17,Sedan,F,DE,1.0


In [4]:
# Copy the dataframe
df2 = data.copy()

In [5]:
def BLP(dataframe):
    # Group the DataFrame by the year
    grouped_data = dataframe.groupby('Year')

    # Define a function to sum the attributes of other models in the same year
    def sum_attributes(row):
        # Filter the DataFrame for the same year excluding the current model
        same_year_data = dataframe[(dataframe['Year'] == row['Year']) & (dataframe['Model'] != row['Model'])]
        
        # Sum the attributes of other models in the same year
        BLP_range = same_year_data['Range'].sum()
        BLP_hp = same_year_data['HP'].sum()
        BLP_chargetime = same_year_data['Chargetime'].sum()
        
        return pd.Series({'BLP_Range': BLP_range, 'BLP_HP': BLP_hp, 'BLP_Chargetime': BLP_chargetime})

    # Apply the function to each row in the DataFrame
    new_columns = dataframe.apply(sum_attributes, axis=1)

    # Add the new columns to the DataFrame
    dataframe['BLP_Range'] = new_columns['BLP_Range']
    dataframe['BLP_HP'] = new_columns['BLP_HP']
    dataframe['BLP_Chargetime'] = new_columns['BLP_Chargetime']

    return dataframe

# Call the function with your input DataFrame
df2 = BLP(df2)

# Print the updated DataFrame with summed attributes
print(df2)


       ID  Year  Sales  Market share Manufacturer   Model  Range  \
8       1  2021    257      0.010319       Aiways      U5    400   
9       1  2022    183      0.005938       Aiways      U5    400   
10      1  2023    177      0.002822       Aiways      U5    400   
21      2  2023      3      0.000048       Aiways      U6    405   
28      3  2019    222      0.040217         Audi  e-tron    375   
...   ...   ...    ...           ...          ...     ...    ...   
1209  189  2023      4      0.000064        Volvo    EX30    475   
1218  190  2021    363      0.014575        Volvo    XC40    457   
1219  190  2022   1020      0.033094        Volvo    XC40    457   
1220  190  2023   1965      0.031327        Volvo    XC40    457   
1231  192  2023      8      0.000128        Voyah    Free    500   

              Price   HP  Chargetime Type Segment Country  const  BLP_Range  \
8     330825.789474  201          34  SUV       C      CH    1.0      22223   
9     330825.789474  201 

In [6]:
# Creating dummies for each segment
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)

# Creating dummies for each year
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)

# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)

In [7]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

In [8]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'HP', 'Chargetime']]
dummies = df2[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                #'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China'
               ]]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['BLP_Range', 'BLP_HP', 'BLP_Chargetime']] 

# Including constant doesnt give full rank.
# Constant can only be in one of the stage to be full rank.

In [9]:
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='unadjusted')

NameError: name 'IV2SLS' is not defined

In [10]:
model.summary

NameError: name 'model' is not defined